In [10]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from model import train_xgb

In [3]:
c = [ 'Entries added in pipe', 'Amount added in pipe',
       'Opportunity Weighted Amount', 'Unique accounts added in pipe',
       'Projects won', 'Amount won', 'Unique accounts won', 'Manager_Util',
       'Senior Associate_Util', 'Senior Manager_Util', 'Associate_Util',
       'Director_Util', 'Director diff from Target', 'SM diff from Target',
       'Associate diff from Target', 'Manager diff from Target',
       'SA diff from Target', 'Fracn of team as As', 'Fracn of team as SAs',
       'Fracn of teams as SMs', 'Fracn of teams as Ms', 'Fracn of teams as Ds',
       'Frac Client Code Manager-Director', 'Frac PD Code Manager-Director',
       'Frac 4 Code Manager-Director', 'Frac PD Code M-',
       'Frac Client Code M-', 'Frac 4 Code M-', 'business_days',
       'log of team size']

In [4]:
outputs_1 = train_xgb(1, ['6.0_2022.0'], c, '/Users/waqarsarguroh/Documents/Revenue EM/features_and_models/')

In [5]:
outputs_2 = train_xgb(2, ['6.0_2022.0', '5.0_2022.0'], c, '/Users/waqarsarguroh/Documents/Revenue EM/features_and_models_2/')


In [6]:
from model import combine_outputs

In [7]:
in_scope = ["ESV - PLS",
"ESV - BCM",
"DDV - IT",
"FS - Banking and Capital Markets",
"DDV - Operations",
"Finance - Enterprise Performance Management"]

In [8]:
to_save = combine_outputs(outputs_1, outputs_2, in_scope)

In [9]:
to_save

,Growth Cell,Year,Month,Revenue Sum,Revenue Forecast,model,variables_explain,ticker_1,ticker_2,ticker,Scope_GCs
0,Cannot Find,2022,5,1.407291e+07,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Cannot Find,2022,6,1.296398e+07,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Cannot Find,2022,7,NaN,455728.3750,t+1,"Senior Manager_Util, Amount won, Associate_Util",1.0,NaN,1.0,0
2,Cannot Find,2022,7,NaN,587911.4375,t+2,"Unique accounts won, Frac Client Code M-, Frac...",NaN,1.0,1.0,0
3,Cannot Find,2022,8,NaN,160595.6875,t+2,"Amount won, Frac Client Code M-, Unique accoun...",NaN,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...
145,"Workforce - Talent, Change & Behaviors",2022,5,6.902114e+05,NaN,NaN,NaN,NaN,NaN,NaN,0
146,"Workforce - Talent, Change & Behaviors",2022,6,5.464224e+05,NaN,NaN,NaN,NaN,NaN,NaN,0
147,"Workforce - Talent, Change & Behaviors",2022,7,NaN,968812.0000,t+1,"Fracn of teams as Ds, Frac Client Code Manager...",0.0,NaN,0.0,0
194,"Workforce - Talent, Change & Behaviors",2022,7,NaN,523892.4375,t+2,"Unique accounts added in pipe, Fracn of team a...",NaN,1.0,1.0,0


# CausalML

In [19]:
from causalml.inference.meta import XGBTRegressor
from xgboost import XGBRegressor

In [20]:
y_causal = all_data_df["Revenue Sum + 1"]
collected_data2 = []
for i in range(len(keep_columns_xgb)):
    treatment_ = all_data_df[keep_columns_xgb[i]]
    X_causal = all_data_df[keep_columns_xgb].drop([keep_columns_xgb[i]], axis = 1)
    treatment = np.array([1 if x>treatment_.median() else 0 for x in treatment_])
    xg = XGBTRegressor(random_state=42)
    te, lb, ub = xg.estimate_ate(X_causal, treatment, y_causal)
    print(keep_columns_xgb[i])
    print(treatment_.median())
    print('Average Treatment Effect (XGBoost): {:.2f} ({:.2f}, {:.2f})'.format(te[0], lb[0], ub[0]))
    collected_data2 = collected_data2 + [[keep_columns_xgb[i], treatment_.median(), te[0]]]

Amount added in pipe
15.150511887854474
Average Treatment Effect (XGBoost): -0.43 (-0.61, -0.24)
Entries added in pipe
5.0
Average Treatment Effect (XGBoost): 0.17 (0.04, 0.30)
Unique accounts added in pipe
4.0
Average Treatment Effect (XGBoost): -0.01 (-0.13, 0.12)
Amount won
14.894722878258218
Average Treatment Effect (XGBoost): 0.18 (0.03, 0.34)
Opportunity Weighted Amount
13.94488921490539
Average Treatment Effect (XGBoost): -0.24 (-0.42, -0.05)
Projects won
5.0
Average Treatment Effect (XGBoost): 0.17 (0.03, 0.32)
Unique accounts won
4.5
Average Treatment Effect (XGBoost): 0.22 (0.06, 0.37)
EM% mean
0.2742874394150146
Average Treatment Effect (XGBoost): 0.25 (0.12, 0.39)
business_days
20.0
Average Treatment Effect (XGBoost): 0.00 (-0.06, 0.06)
SA diff from Target
-92.26666666666667
Average Treatment Effect (XGBoost): 0.16 (0.08, 0.23)
Associate diff from Target
-39.75
Average Treatment Effect (XGBoost): 0.18 (0.11, 0.24)
SM diff from Target
-86.6
Average Treatment Effect (XGBoost)

In [22]:
import pandas as pd

In [23]:
f = pd.DataFrame(collected_data2)

In [24]:
f.columns = ['Feature', 'Median', 'ATE']

In [25]:
f['Size of Effect'] = f['ATE'].apply(lambda x: (np.exp(np.abs(x)) - 1)*100)

In [26]:
f.sort_values('ATE', ascending = False)

,Feature,Median,ATE,Size of Effect
24,log of team size,4.226834,1.182198,226.153377
16,Fracn of teams as Ms,19.899875,0.437495,54.882232
19,Frac 4 Code Manager-Director,0.002520,0.394332,48.339332
22,Fracn of team as SAs,27.906977,0.383705,46.771235
20,Frac 4 Code M-,0.001670,0.304076,35.537159
7,EM% mean,0.274287,0.254058,28.924614
18,Frac PD Code M-,0.002167,0.219895,24.594592
6,Unique accounts won,4.500000,0.215002,23.986499
3,Amount won,14.894723,0.183366,20.125421
10,Associate diff from Target,-39.750000,0.176708,19.328311


In [28]:
pd.DataFrame(all_data_df.columns.values)

,0
0,Entries added in pipe
1,Opportunity Weighted Amount
2,Amount added in pipe
3,Unique accounts added in pipe
4,Projects won
5,Unique accounts won
6,Amount won
7,Director_Util
8,Associate_Util
9,Senior Manager_Util
